### Imports

In [10]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np
import math

In [11]:
# https://naif.jpl.nasa.gov/naif/aboutspice.html
import spiceypy as spice

### Constants

In [12]:
DEG = 180.0/np.pi #degrees #spice angles are in radians
MU_SUN = 1.32712440018e11  #km^3/s^2 #standard gravitational parameter of Sun

### Get Data

In [13]:
# https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/
# https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/

def load_kernels():
    spice.furnsh("../data/naif0012.tls")      # Leap seconds - conversion of earth time to ephemeris time
    spice.furnsh("../data/de440s.bsp")        # Ephemeris data - position and velocities
    spice.furnsh("../data/pck00011.tpc")      # Planetary constants - radius, spin axes, rotation rates etc.

### Get Angle between 2 Vectors

In [14]:
def get_angle_between_vectors(v1, v2):
    v1 = np.array(v1, dtype=float)
    v2 = np.array(v2, dtype=float)
    dot = np.dot(v1, v2)
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    if norm1 == 0 or norm2 == 0:
        raise ValueError("One of the vectors has zero length")
    cos_theta = dot / (norm1 * norm2)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    return np.arccos(cos_theta)
    # return spice.vsep(v1, v2)

### Get Angular Distance

In [15]:
# Get angular radius (how big a body of given radius looks from given distance) - values clamped [0,1]
def get_angular_radius(radius_km, dist_km):
    x = max(0.0, min(1.0, radius_km / max(dist_km, 1e-9)))
    return math.asin(x)

### Get Radius

In [16]:
# https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/IDL/icy/cspice_bodvrd.html
def get_mean_radius_km(body_name):
    radii = spice.bodvrd(body_name.upper(), "RADII", 3)[1]
    return float(np.mean(radii))

### Get State Vector

In [17]:
# codes for solar system bodies
# 10 - Sun, 399 - Earth, 301 - Moon
# https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/req/naif_ids.html#Planets%20and%20Satellites

def get_state_vec(targ, obs, et, ref="J2000"):
    state, lt = spice.spkgeo(targ=targ, et=et, ref=ref, obs=obs)
    return np.array(state[:3])

### Get Angular Separation  

In [ ]:
def solar_moon_separation_over_range(start_date, end_date, step_hours=6):
    ets, seps_deg, dates = [], [], []
    t = start_date
    while t <= end_date:
        et = spice.utc2et(t.strftime("%Y-%m-%dT%H:%M:%S"))
        r_es = get_state_vec(10, 399, et)     # target sun  10 wrt Earth 399
        r_em = get_state_vec(301, 399, et)    # target moon 301 wrt Earth 399
        sep = get_angle_between_vectors(r_es, r_em) * DEG 
        ets.append(et); seps_deg.append(sep); dates.append(t)
        t += timedelta(hours=step_hours)
    return np.array(ets), np.array(seps_deg), dates

### Get Eclipse overlap 

In [ ]:
def circle_overlap_fraction(target_r, eclipsing_body_r, center_dist, eps=1e-12):
    r1, r2, d = float(target_r), float(eclipsing_body_r), float(center_dist)

    #distance between circles centers is greater than sum of radius (no touch)
    if d >= r1 + r2: 
        return 0.0
    #distance between circles is greater than diff of radius (total eclipse)
    # target body inside eclipsing body - eclipsing body is larger
    if d <= abs(r2 - r1) and r2 >= r1:
        return 1.0
    #distance between circles is less than diff of radius (partial eclipse on target)
    #eclipsing body inside target - eclipsing body is smaller 
    if d <= abs(r2 - r1) and r2 < r1:
        return (math.pi * r2**2) / (math.pi * r1**2)

    # Partial overlap
    # law of cosines (angles at which circles intersect))
    c1 = (d**2 + r1**2 - r2**2) / (2*d*r1 + eps)
    c2 = (d**2 + r2**2 - r1**2) / (2*d*r2 + eps)
    c1 = max(-1.0, min(1.0, c1)) # clip to [-1,1] to avoid fp issues
    c2 = max(-1.0, min(1.0, c2))

    # area of circular segments
    a1 = r1**2 * math.acos(c1)
    a2 = r2**2 * math.acos(c2)
    # area of triangle formed by circles centers and intersection points (Heron's)
    a3 = 0.5 * math.sqrt(max(0.0, (-d+r1+r2)*(d+r1-r2)*(d-r1+r2)*(d+r1+r2)))

    overlap_area = a1 + a2 - a3
    target_area = math.pi * r1**2
    return max(0.0, min(1.0, overlap_area / (target_area + eps)))

# Get Gamma (offset from earth's center)

In [ ]:
def compute_gamma_at_time(et):
    r_es = get_state_vec(10,  399, et)   # Sun wrt Earth
    r_em = get_state_vec(301, 399, et)   # Moon wrt Earth

    # Shadow axis direction at the Moon toward the sun in the Earth frame
    u = r_es - r_em                      # vector Moon - Sun in Earth frame
    u = u / np.linalg.norm(u)            # unit direction

    # Closest vector from the line (through r_em along u) to Earth's center (origin):
    # v = - u × (r_em × u)
    v = -np.cross(u, np.cross(u, r_em))
    # Sign: north (+) if v has positive component along J2000 +Z (Earth's north)
    sign = np.sign(np.dot(np.array([0.0, 0.0, 1.0]), v))

    Re_eq = get_mean_radius_km(body_name="EARTH")    
    gamma = sign * (np.linalg.norm(v) / Re_eq)
    return gamma

# Get Metrics

In [ ]:
def get_solar_metrics_at_time(et):
    # radius of sun and moon in km
    R_sun  = get_mean_radius_km("SUN")
    R_moon = get_mean_radius_km("MOON")
    R_earth = get_mean_radius_km("EARTH")

    #position vectors for sun and moon wrt earth
    r_es = get_state_vec(10, 399, et)
    r_em = get_state_vec(301, 399, et)

    # distance of sun and moon from earth
    d_es = np.linalg.norm(r_es) 
    d_em = np.linalg.norm(r_em)  

    # apparent angular radius (size) of sun and moon from earth
    a_sun  = get_angular_radius(R_sun,  d_es)
    a_moon = get_angular_radius(R_moon, d_em)

    # apparent distance between sun and moon from earth
    sep = get_angle_between_vectors(r_es, r_em) #Only from geocenter

    # Anywhere from earth - Lunar horizontal parallax ≈ asin(R_earth / d_em)  (~57 arcmin)
    pi_E = math.asin(min(1.0, R_earth / max(d_em, 1e-9)))
    # We can reduce the geocenter separation by at most pi_E by choosing an optimalobserver on Earth's surface
    sep_global = max(0.0, sep - pi_E)

    # fraction of Sun's disk covered (planar small-angle approx)
    frac = circle_overlap_fraction(a_sun, a_moon, sep_global)

    if sep_global <= abs(a_moon - a_sun):
        if a_moon > a_sun: #if moon looks bigger than sun
            kind = "total"
        elif a_moon < a_sun: #if moon looks smaller than sun
            kind = "annular"
        else:
            kind = "exactly equal angular sizes"
    elif frac > 0: # partial eclipse
        kind = "partial"
    else:
        kind = "no eclipse"

    gamma = compute_gamma_at_time(et)

    return {
        "sun_obscuration_fraction": frac,     
        "sep_deg": sep * DEG,
        "alpha_sun_deg": a_sun * DEG,
        "alpha_moon_deg": a_moon * DEG,
        "classification": kind,
        "gamma": gamma
    }


In [ ]:
def get_lunar_metrics_at_time(et):
    # radius of sun and moon in km
    R_earth = get_mean_radius_km("EARTH")
    R_sun   = get_mean_radius_km("SUN")
    R_moon  = get_mean_radius_km("MOON")

    # position vectors for sun and earth wrt moon
    r_em = get_state_vec(301, 399, et)  # Moon wrt Earth
    r_es = get_state_vec(10,  399, et)  # Sun  wrt Earth
    r_ms = get_state_vec(10, 301, et)   # Sun wrt Moon
    d_ms = np.linalg.norm(r_ms)
    d_me = np.linalg.norm(r_em)

    # apparent angular radius (size) of sun and earth from moon
    a_E = get_angular_radius(R_earth, d_me)
    a_S = get_angular_radius(R_sun, d_ms)

    # shadow angular radii (radians) (angular size of earth - sun)
    a_umb = max(0.0, a_E - a_S)
    a_pen = a_E + a_S
    # in km
    r_umb = d_me * math.tan(a_umb)
    r_pen = d_me * math.tan(a_pen)

    # offset between earth and sun poditiond from moon (radians)
    theta = get_angle_between_vectors(r_em, -r_ms)
    # in km
    s = d_me * theta

    # fractions of the Moon's disk covered
    frac_umb = circle_overlap_fraction(R_moon, r_umb, s) if r_umb > 0 else 0.0
    frac_pen = circle_overlap_fraction(R_moon, r_pen, s)
    # along diameter - negative umbral magnitude => no umbral contact; >1 => total (Moon fully within umbra)
    umbral_mag    = (r_umb + R_moon - s) / (2.0 * R_moon)
    penumbral_mag = (r_pen + R_moon - s) / (2.0 * R_moon)

    if frac_umb >= 1.0:
        kind = "total lunar eclipse (Moon fully in umbra)"
    elif frac_umb > 0:
        kind = "partial lunar eclipse (Moon partially in umbra)"
    elif frac_pen > 0:
        kind = "penumbral lunar eclipse (Moon only in penumbra)"
    else:
        kind = "no lunar eclipse"

    return {
        "umbra_fraction_of_moon": frac_umb,   
        "penumbra_fraction_of_moon": frac_pen,
        "theta_deg": theta * DEG,
        "alpha_earth_deg": a_E * DEG,
        "alpha_sun_deg": a_S * DEG,
        "classification": kind,
        "umbral_magnitude": umbral_mag,
        "penumbral_magnitude": penumbral_mag,
    }


# Get Eclipse Candidate in a time period

In [ ]:
def scan_solar_eclipses(start_dt, end_dt, step_size, min_obsc=0.01):
    t = start_dt
    events = []
    active = False
    current = None
    while t <= end_dt:
        et = spice.utc2et(t.strftime("%Y-%m-%dT%H:%M:%S"))
        m = get_solar_metrics_at_time(et)
        if m["sun_obscuration_fraction"] >= min_obsc:
            if not active:
                active = True
                current = {"start": t, "peak": (t, m), "peak_val": m["sun_obscuration_fraction"], "min_gamma": (t, m["gamma"], abs(m["gamma"]))}
            else:
                if m["sun_obscuration_fraction"] > current["peak_val"]:
                    current["peak"] = (t, m)
                    current["peak_val"] = m["sun_obscuration_fraction"]
                if abs(m["gamma"]) < current["min_gamma"][2]:
                    current["min_gamma"] = (t, m["gamma"], abs(m["gamma"]))
        elif active:
            current["end"] = t
            events.append(current)
            active = False
        t += step_size
    if active:
        current["end"] = end_dt
        events.append(current)
    return events

In [ ]:
def scan_lunar_eclipses(start_dt, end_dt, step_size, min_umb_or_pen=0.01):
    t = start_dt
    events = []
    active = False
    current = None
    while t <= end_dt:
        et = spice.utc2et(t.strftime("%Y-%m-%dT%H:%M:%S"))
        m = get_lunar_metrics_at_time(et)
        measure = max(m["umbra_fraction_of_moon"], m["penumbra_fraction_of_moon"])
        if measure >= min_umb_or_pen:
            if not active:
                active = True
                current = {"start": t, "peak": (t, m), "peak_val": measure}
            else:
                if measure > current["peak_val"]:
                    current["peak"] = (t, m)
                    current["peak_val"] = measure
        elif active:
            current["end"] = t
            events.append(current)
            active = False
        t += step_size
    if active:
        current["end"] = end_dt
        events.append(current)
    return events

# RUN

In [ ]:
load_kernels()
start = datetime(2024, 1, 1)
end   = datetime(2026, 12, 31)
step_size_solar = timedelta(minutes=5)
step_size_lunar = timedelta(hours=1)

solar_events = scan_solar_eclipses(start, end, step_size_solar, min_obsc=0.01)
print("SOLAR ECLIPSE CANDIDATES:")
for ev in solar_events:
    peak_t, peak_m = ev["peak"]
    print(f"{ev['start']} → {ev['end']} | peak @ {peak_t} | obsc={peak_m['sun_obscuration_fraction']:.3f} | {peak_m['classification'] }| gamma={peak_m['gamma']:.3f}")

lunar_events = scan_lunar_eclipses(start, end, step_size_lunar, min_umb_or_pen=0.01)
print("LUNAR ECLIPSE CANDIDATES:")
for ev in lunar_events:
    peak_t, peak_m = ev["peak"]
    print(f"{ev['start']} → {ev['end']} | peak @ {peak_t} | "
          f"umbra={peak_m['umbra_fraction_of_moon']:.3f}, pen={peak_m['penumbra_fraction_of_moon']:.3f} | {peak_m['classification']}| umbral_mag={peak_m['umbral_magnitude']:.3f}, penumbral_mag={peak_m['penumbral_magnitude']:.3f}")



SOLAR ECLIPSE CANDIDATES (geocenter):
2024-04-08 17:40:00 → 2024-04-08 19:00:00 | peak @ 2024-04-08 18:20:00 | obsc=0.252 | partial| gamma=0.345
2024-10-02 18:00:00 → 2024-10-02 19:40:00 | peak @ 2024-10-02 18:45:00 | obsc=0.246 | partial| gamma=-0.352

LUNAR ECLIPSE CANDIDATES:
2024-03-25 05:00:00 → 2024-03-25 09:30:00 | peak @ 2024-03-25 07:15:00 | umbra=0.000, pen=0.970 | penumbral lunar eclipse (Moon only in penumbra)| umbral_mag=-0.150, penumbral_mag=0.935
2024-09-18 00:50:00 → 2024-09-18 04:45:00 | peak @ 2024-09-18 02:35:00 | umbra=0.020, pen=1.000 | partial lunar eclipse (Moon partially in umbra)| umbral_mag=0.058, penumbral_mag=1.008


# Visualization

In [54]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

DEG = 180.0 / np.pi
REF_HELIO = "ECLIPJ2000"
REF_GEO   = "J2000"

# ---------- window ----------
START = datetime(2024, 4, 8)
END   = datetime(2024, 4, 8)
STEP  = timedelta(minutes=15)

AU_KM      = 149_597_870.7
SYS_PAD    = 1.2 * AU_KM    # top system view
ZOOM_PAD   = 600_000.0      # bottom Earth–Moon zoom half-span (km)
LIM_LUNAR  = 10_000         # +/- km @ Moon for lunar panel (keep circular look)
SOLAR_XY   = (-2.0, 2.0, -1.4, 1.4)  # degrees, fixed axes for solar panel

# ---------- utils ----------
def utc_to_et(dt):
    import spiceypy as spice
    return spice.utc2et(dt.strftime("%Y-%m-%dT%H:%M:%S"))

def circle_poly(center, radius, n=240):
    cx, cy = center
    th = np.linspace(0, 2*np.pi, n, endpoint=True)
    return cx + radius*np.cos(th), cy + radius*np.sin(th)

def concentric_disks(center, radius, n=180, steps=4, color="rgba(0,0,0,1)", fade_to=0.4):
    """
    Build a subtle radial 'sphere' using concentric filled polygons.
    steps: number of rings (outer -> inner). fade_to controls inner opacity.
    """
    cx, cy = center
    traces = []
    for k in range(steps, 0, -1):
        r = radius * (0.35 + 0.65 * k/steps)       # shrink slightly toward center
        op = (fade_to + (1.0 - fade_to) * (k/steps))
        rgba = color.replace("1)", f"{op})")
        x, y = circle_poly((cx, cy), r, n=n)
        traces.append(go.Scatter(x=x, y=y, mode="lines", fill="toself",
                                 line=dict(width=0.8, color="rgba(0,0,0,0.6)"),
                                 fillcolor=rgba, showlegend=False))
    return traces

def heliocentric_positions(et):
    import spiceypy as spice
    r_e_sun, _ = spice.spkgeo(targ=399, et=et, ref=REF_HELIO, obs=10)
    r_m_ecl, _ = spice.spkgeo(targ=301, et=et, ref=REF_HELIO, obs=399)
    r_es = np.array(r_e_sun[:3], float)
    r_ms = r_es + np.array(r_m_ecl[:3], float)
    r_sun_geo = get_state_vec(10,  399, et)    # Sun wrt Earth (J2000)
    r_moon_geo= get_state_vec(301, 399, et)    # Moon wrt Earth (J2000)
    return r_es, r_ms, r_sun_geo, r_moon_geo

def lunar_shadow_at_moon(et):
    R_earth = get_mean_radius_km("EARTH")
    R_sun   = get_mean_radius_km("SUN")
    R_moon  = get_mean_radius_km("MOON")
    r_me = -get_state_vec(301, 399, et)      # Earth wrt Moon
    r_ms =  get_state_vec(10,  301, et)      # Sun wrt Moon
    d_me, d_ms = np.linalg.norm(r_me), np.linalg.norm(r_ms)
    a_E = get_angular_radius(R_earth, d_me)
    a_S = get_angular_radius(R_sun,   d_ms)
    a_umb = max(0.0, a_E - a_S)
    a_pen = a_E + a_S
    r_umb = d_me * math.tan(a_umb)
    r_pen = d_me * math.tan(a_pen)
    theta = get_angle_between_vectors(r_me, -r_ms)
    s = d_me * theta
    return R_moon, r_umb, r_pen, s

def circle_poly(center, radius, n=360):  # increase resolution
    cx, cy = center
    th = np.linspace(0, 2*np.pi, n, endpoint=False)
    return cx + radius*np.cos(th), cy + radius*np.sin(th)

# ---------- build timeline ----------
times = []
t = START
while t <= END:
    times.append(t); t += STEP
N = len(times)

# ---------- precompute ----------
earth_xy = np.zeros((N,2))
moon_xy  = np.zeros((N,2))
moon_geo = np.zeros((N,3))

sunR_deg  = np.zeros(N); moonR_deg = np.zeros(N); sep_deg = np.zeros(N)
cover_pct = np.zeros(N); classif   = [""]*N

moon_Rkm  = np.zeros(N); umb_km  = np.zeros(N); pen_km = np.zeros(N); s_km = np.zeros(N)
umb_frac  = np.zeros(N); pen_frac = np.zeros(N)

for i, dt in enumerate(times):
    et = utc_to_et(dt)
    r_es, r_ms, r_sun_geo, r_moon_geo = heliocentric_positions(et)
    earth_xy[i] = r_es[:2]; moon_xy[i] = r_ms[:2]; moon_geo[i] = r_moon_geo

    d_es = np.linalg.norm(r_sun_geo); d_em = np.linalg.norm(r_moon_geo)
    sunR_deg[i]  = get_angular_radius(get_mean_radius_km("SUN"),  d_es) * DEG
    moonR_deg[i] = get_angular_radius(get_mean_radius_km("MOON"), d_em) * DEG
    sep_deg[i]   = get_angle_between_vectors(r_sun_geo, r_moon_geo) * DEG

    frac = circle_overlap_fraction(math.radians(sunR_deg[i]),
                                   math.radians(moonR_deg[i]),
                                   math.radians(sep_deg[i]))
    cover_pct[i] = 100.0 * frac
    classif[i]   = get_solar_metrics_at_time(et)["classification"]

    Rm, ru, rp, s = lunar_shadow_at_moon(et)
    moon_Rkm[i], umb_km[i], pen_km[i], s_km[i] = Rm, ru, rp, s
    lm = get_lunar_metrics_at_time(et)
    umb_frac[i] = lm["umbra_fraction_of_moon"]   * 100.0
    pen_frac[i] = lm["penumbra_fraction_of_moon"]* 100.0

# ---------- layout: 3 rows ----------
# row 1: system view (full width)
# row 2: solar (left), lunar (right)
# row 3: Earth–Moon zoom (full width)
fig = make_subplots(
    rows=3, cols=2,
    specs=[
        [{"type":"scatter", "colspan":2}, None],
        [{"type":"scatter"}, {"type":"scatter"}],
        [{"type":"scatter", "colspan":2}, None],
    ],
    vertical_spacing=0.10, horizontal_spacing=0.08,
    subplot_titles=(
        "Top‑down ecliptic view (Sun at origin)",
        "Solar eclipse (apparent disks, deg) — Sun fixed",
        "Lunar eclipse (Moon vs Earth's shadow @ Moon, km) — Moon fixed",
        "Earth–Moon zoom (geocentric, km)"
    )
)

# --- Row 1: system view ---
fig.add_trace(go.Scatter(x=[0], y=[0], mode="markers",
                         marker=dict(size=18, color="gold"), name="Sun"), row=1, col=1)
fig.add_trace(go.Scatter(x=[], y=[], mode="lines", name="Earth path", line=dict(width=1.2)), row=1, col=1)
fig.add_trace(go.Scatter(x=[], y=[], mode="markers",
                         marker=dict(size=10, color="royalblue"), name="Earth"), row=1, col=1)
fig.add_trace(go.Scatter(x=[], y=[], mode="lines", name="Moon path (heliocentric)", line=dict(width=1, color="gray")),
              row=1, col=1)
fig.add_trace(go.Scatter(x=[], y=[], mode="markers",
                         marker=dict(size=8, color="dimgray"), name="Moon"), row=1, col=1)

fig.update_xaxes(range=[-SYS_PAD, SYS_PAD], title="km", row=1, col=1)
fig.update_yaxes(range=[-SYS_PAD, SYS_PAD], title="km", row=1, col=1)
fig.update_yaxes(scaleanchor="x", scaleratio=1, row=1, col=1)

# --- Row 2 left: SOLAR (Sun fixed @ origin, Moon slides). fixed axes so circles stay circles
fig.add_trace(go.Scatter(x=[], y=[], mode="lines", fill="toself",
                         name="Sun disk", line=dict(color="rgba(0,0,0,0.5)", width=1),
                         fillcolor="rgba(255,215,0,0.95)"),
              row=2, col=1)
# we’ll layer a subtle gradient with extra rings
fig.add_trace(go.Scatter(x=[], y=[], mode="lines", fill="toself",
                         name="Moon disk (shadow)", line=dict(color="rgba(0,0,0,0.6)", width=1),
                         fillcolor="rgba(0,0,0,0.9)"),
              row=2, col=1)
fig.add_trace(go.Scatter(x=[None], y=[None], mode="text", showlegend=False), row=2, col=1)

x0, x1, y0, y1 = SOLAR_XY
fig.update_xaxes(range=[x0, x1], title="degrees", row=2, col=1)
fig.update_yaxes(range=[y0, y1], showticklabels=False, row=2, col=1)
fig.update_yaxes(scaleanchor="x", scaleratio=1, row=2, col=1)

# --- Row 2 right: LUNAR (Moon fixed @ origin; Earth shadow slides). fixed axes too
fig.add_trace(go.Scatter(x=[], y=[], mode="lines", fill="toself",
                         name="Earth penumbra", line=dict(color="gray", width=1),
                         fillcolor="rgba(120,120,120,0.35)"),
              row=2, col=2)
fig.add_trace(go.Scatter(x=[], y=[], mode="lines", fill="toself",
                         name="Earth umbra", line=dict(color="black", width=1),
                         fillcolor="rgba(0,0,0,0.55)"),
              row=2, col=2)
# Moon gradient (silver)
fig.add_trace(go.Scatter(x=[], y=[], mode="lines", fill="toself",
                         name="Moon", line=dict(color="rgba(0,0,0,0.6)", width=1),
                         fillcolor="silver"),
              row=2, col=2)
fig.add_trace(go.Scatter(x=[None], y=[None], mode="text", showlegend=False), row=2, col=2)

fig.update_xaxes(range=[-LIM_LUNAR, LIM_LUNAR], title="km @ Moon", row=2, col=2)
fig.update_yaxes(range=[-LIM_LUNAR, LIM_LUNAR], showticklabels=False, row=2, col=2)
fig.update_yaxes(scaleanchor="x", scaleratio=1, row=2, col=2)

# --- Row 3: Earth–Moon zoom (true 2D position)
fig.add_trace(go.Scatter(x=[], y=[], mode="lines", name="Moon orbit (geocentric)", line=dict(width=1, dash="dot")),
              row=3, col=1)
fig.add_trace(go.Scatter(x=[], y=[], mode="markers+text", name="Earth (center)",
                         marker=dict(size=12, color="royalblue"), text=["Earth"], textposition="top center"),
              row=3, col=1)
fig.add_trace(go.Scatter(x=[], y=[], mode="markers", name="Moon (geo)",
                         marker=dict(size=10, color="dimgray")),
              row=3, col=1)

# fig.update_xaxes(range=[-ZOOM_PAD, ZOOM_PAD], title="km (geocentric)", row=3, col=1)
# fig.update_yaxes(range=[-ZOOM_PAD, ZOOM_PAD], showticklabels=False, row=3, col=1)
fig.update_xaxes(range=[-ZOOM_PAD, ZOOM_PAD], fixedrange=True, title="km (geocentric)", row=3, col=1)
fig.update_yaxes(range=[-ZOOM_PAD, ZOOM_PAD], fixedrange=True, showticklabels=False, row=3, col=1)

fig.update_yaxes(scaleanchor="x", scaleratio=1, row=3, col=1)

# ---------- frames ----------
frames = []
for i, dt in enumerate(times):
    # system view trails
    ex, ey = earth_xy[:i+1,0], earth_xy[:i+1,1]
    mx, my = moon_xy[:i+1,0],  moon_xy[:i+1,1]

    # solar panel
    # Sun disk (fixed)
    sun_x, sun_y = circle_poly((0.0, 0.0), sunR_deg[i], n=240)
    # Moon shadow (placed at offset based on angular separation)
    moon_x, moon_y = circle_poly((sep_deg[i], 0.0), moonR_deg[i], n=240)
    # Use darker sun background proportional to coverage
    sun_opacity = 1.0 - 0.75 * (cover_pct[i]/100)
    sun_color = f"rgba(255,215,0,{sun_opacity:.2f})"
    solar_text = f"Sun covered: {cover_pct[i]:.1f}%<br>{classif[i]}<br>{dt.strftime('UTC %Y-%m-%d %H:%M')}"

    # lunar panel
    # Earth's shadow centers at (s_km[i], 0)
    pen_x, pen_y = circle_poly((s_km[i], 0.0), pen_km[i])
    umb_x, umb_y = circle_poly((s_km[i], 0.0), umb_km[i])
    # Moon fixed at origin
    moon_km_x, moon_km_y  = circle_poly((0.0, 0.0), moon_Rkm[i])
    # Apply opacity based on coverage
    umb_opacity = 0.4 + 0.6 * (umb_frac[i]/100)
    pen_opacity = 0.2 + 0.4 * (pen_frac[i]/100)
    umb_color = f"rgba(0,0,0,{umb_opacity:.2f})"
    pen_color = f"rgba(120,120,120,{pen_opacity:.2f})"
    lunar_text = f"Umbra cover: {umb_frac[i]:.1f}%<br>Penumbral cover: {pen_frac[i]:.1f}%"

    # bottom zoom: ring + true Moon position
    r_geo = np.linalg.norm(moon_geo[i])
    th = np.linspace(0, 2*np.pi, 256)
    ring_x = r_geo*np.cos(th); ring_y = r_geo*np.sin(th)
    moon_geo_xy = (moon_geo[i,0], moon_geo[i,1])

    # add subtle gradients as extra layers (keeps circular look even when zooming)
    sun_grad = concentric_disks((0.0, 0.0), sunR_deg[i], steps=3,
                                color="rgba(255,215,0,1)", fade_to=0.55)
    moon_grad = concentric_disks((sep_deg[i], 0.0), moonR_deg[i], steps=2,
                                 color="rgba(0,0,0,1)", fade_to=0.75)
    moon_silver_grad = concentric_disks((0.0, 0.0), moon_Rkm[i], steps=2,
                                        color="rgba(192,192,192,1)", fade_to=0.7)
    
    frames.append(go.Frame(
        name=str(i),
        data=[
            # row1 system
            go.Scatter(x=[0], y=[0]),
            go.Scatter(x=ex, y=ey),
            go.Scatter(x=[earth_xy[i,0]], y=[earth_xy[i,1]]),
            go.Scatter(x=mx, y=my),
            go.Scatter(x=[moon_xy[i,0]], y=[moon_xy[i,1]]),

            # row2 solar (base disks)
            # Sun as marker (with dynamic opacity)
            go.Scatter(
                x=[0.0], y=[0.0],
                mode='markers',
                marker=dict(
                    size=sunR_deg[i]*400,  # scaling factor to convert degrees to px-ish (tune this)
                    color=f'rgba(255,215,0,{sun_opacity:.2f})',
                    line=dict(width=1, color="gold"),
                    symbol='circle'
                ),
                showlegend=False
            ),
            # Moon as marker (shadow over Sun)
            go.Scatter(
                x=[sep_deg[i]], y=[0.0],
                mode='markers',
                marker=dict(
                    size=moonR_deg[i]*400,  # same scale
                    color='black',
                    line=dict(width=1, color="black"),
                    symbol='circle'
                ),
                showlegend=False
            ),

            go.Scatter(
                x=[-1.8], y=[1.2],  # inside axes
                mode="text",
                text=[solar_text],
                textposition="top right",
                showlegend=False
            ),

            # row2 lunar (shadows + moon)
            # Penumbra (Earth's light shadow)
            go.Scatter(
                x=[s_km[i]], y=[0.0],
                mode='markers',
                marker=dict(
                    size=pen_km[i]*0.02,  # scale km to pixels
                    color=pen_color,
                    line=dict(width=0.5, color='gray'),
                    symbol='circle'
                ),
                showlegend=False
            ),
            
            # Umbra (Earth's full shadow)
            go.Scatter(
                x=[s_km[i]], y=[0.0],
                mode='markers',
                marker=dict(
                    size=umb_km[i]*0.02,  # scale km to pixels
                    color=umb_color,
                    line=dict(width=0.5, color='black'),
                    symbol='circle'
                ),
                showlegend=False
            ),
            
            # Moon
            go.Scatter(
                x=[0.0], y=[0.0],
                mode='markers',
                marker=dict(
                    size=moon_Rkm[i]*0.02,
                    color='silver',
                    line=dict(width=1, color='dimgray'),
                    symbol='circle'
                ),
                showlegend=False
            ),

            go.Scatter(
                x=[-LIM_LUNAR*0.9], y=[LIM_LUNAR*0.9],
                mode="text",
                text=[lunar_text],
                textposition="top right",
                showlegend=False
            ),


            # row3 zoom

            go.Scatter(x=ring_x, y=ring_y),
            go.Scatter(x=[0], y=[0]),
            go.Scatter(x=[moon_geo_xy[0]], y=[moon_geo_xy[1]]),


        ],
        # add the gradient layers to the appropriate subplots
        traces=None,  # we’re adding via layout updaters below
        layout=go.Layout(
            # add extra gradient layers (solar)
            # NOTE: we can't place subplots per-trace here easily; so we rely on
            # adding them as additional data via extend in data[]:
        )
    ))

    # Append gradient layers to the same frame data (after base disks)
    frames[-1].data = tuple(list(frames[-1].data) + sun_grad + moon_grad + moon_silver_grad)

# ---------- controls ----------
sliders = [dict(
    steps=[dict(method="animate",
                args=[[str(k)], dict(mode="immediate",
                                     frame=dict(duration=120, redraw=True),
                                     transition=dict(duration=0))],
                label=times[k].strftime("%Y-%m-%d")) for k in range(N)],
    transition=dict(duration=0),
    x=0.10, y=0.02, xanchor="left", yanchor="top",
    len=0.80, currentvalue=dict(prefix="Time: ", visible=True)
)]
updatemenus = [dict(
    type="buttons", direction="left",
    x=0.10, y=1.06, xanchor="left", yanchor="top",
    buttons=[
        dict(label="Play",  method="animate",
             args=[None, dict(frame=dict(duration=120, redraw=True),
                              transition=dict(duration=0),
                              fromcurrent=True, mode="immediate")]),
        dict(label="Pause", method="animate",
             args=[[None], dict(frame=dict(duration=0, redraw=False),
                                transition=dict(duration=0),
                                mode="immediate")])
    ]
)]

fig.update_layout(
    title="Orbital Motion + Solar & Lunar Eclipses (rows split; disks stay circular)",
    height=950, sliders=sliders, updatemenus=updatemenus, showlegend=True
)

fig.update(frames=frames)
fig.show()

# Vis2

In [55]:
# --- hoist imports/util trig ---
import numpy as np, math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# unit circles reused everywhere (no per-frame trig)
TH_SOL  = np.linspace(0, 2*np.pi, 240, endpoint=False)
UCOS_SOL, USIN_SOL = np.cos(TH_SOL), np.sin(TH_SOL)
TH_RING = np.linspace(0, 2*np.pi, 256, endpoint=False)
RCOS, RSIN = np.cos(TH_RING), np.sin(TH_RING)

def bounds_from_center_r(cx, cy, r):
    return cx - r, cy - r, cx + r, cy + r

# --- build timeline as before ---
times = []
t = START
while t <= END:
    times.append(t); t += STEP
N = len(times)

# --- pre-create traces so frames just update them by index ---
fig = make_subplots(
    rows=3,
    cols=2,
    specs=[
        [ {"type": "scatter", "colspan": 2}, None],   # (1,1) spans both columns
        [ {"type": "scatter"}, {"type": "scatter"}],  # (2,1) and (2,2)
        [ {"type": "scatter", "colspan": 2}, None],   # (3,1) spans both columns
    ],
    vertical_spacing=0.10,
    horizontal_spacing=0.08,
    subplot_titles=(
        "Top-down ecliptic view (Sun at origin)",
        "Solar eclipse (apparent disks, deg) — Sun fixed",
        "Lunar eclipse (Moon vs Earth's shadow @ Moon, km) — Moon fixed",
        "Earth–Moon zoom (geocentric, km)"
    )
)

# Row 1 (system) – trails + markers
i_sys_sun     = len(fig.data); fig.add_trace(go.Scatter(x=[0], y=[0], mode="markers", marker=dict(size=18, color="gold"), name="Sun"), row=1, col=1)
i_sys_epath   = len(fig.data); fig.add_trace(go.Scatter(x=[], y=[], mode="lines",  name="Earth path", line=dict(width=1.2)), row=1, col=1)
i_sys_earth   = len(fig.data); fig.add_trace(go.Scatter(x=[], y=[], mode="markers",marker=dict(size=10, color="royalblue"), name="Earth"), row=1, col=1)
i_sys_mpath   = len(fig.data); fig.add_trace(go.Scatter(x=[], y=[], mode="lines",  name="Moon path (heliocentric)", line=dict(width=1, color="gray")), row=1, col=1)
i_sys_moon    = len(fig.data); fig.add_trace(go.Scatter(x=[], y=[], mode="markers",marker=dict(size=8, color="dimgray"), name="Moon"), row=1, col=1)

# Row 2 (solar) – two markers + text
i_sol_sun     = len(fig.data); fig.add_trace(go.Scatter(x=[0], y=[0], mode="markers", showlegend=False), row=2, col=1)
i_sol_moon    = len(fig.data); fig.add_trace(go.Scatter(x=[0], y=[0], mode="markers", showlegend=False), row=2, col=1)
i_sol_text    = len(fig.data); fig.add_trace(go.Scatter(x=[None], y=[None], mode="text", showlegend=False), row=2, col=1)


# Row 2 (lunar) – pen/umbra & Moon with markers + text
i_lun_pen     = len(fig.data); fig.add_trace(go.Scatter(x=[0], y=[0], mode="markers", showlegend=False), row=2, col=2)
i_lun_umb     = len(fig.data); fig.add_trace(go.Scatter(x=[0], y=[0], mode="markers", showlegend=False), row=2, col=2)
i_lun_moon    = len(fig.data); fig.add_trace(go.Scatter(x=[0], y=[0], mode="markers", showlegend=False), row=2, col=2)
i_lun_text    = len(fig.data); fig.add_trace(go.Scatter(x=[None], y=[None], mode="text", showlegend=False), row=2, col=2)

# Row 3 (geo zoom) – ring + Earth + Moon
i_geo_ring    = len(fig.data); fig.add_trace(go.Scatter(x=[], y=[], mode="lines", name="Moon orbit (geocentric)", line=dict(width=1, dash="dot")), row=3, col=1)
i_geo_earth   = len(fig.data); fig.add_trace(go.Scatter(x=[0], y=[0], mode="markers+text", name="Earth (center)",
                                                        marker=dict(size=12, color="royalblue"), text=["Earth"], textposition="top center"), row=3, col=1)
i_geo_moon    = len(fig.data); fig.add_trace(go.Scatter(x=[0], y=[0], mode="markers", name="Moon (geo)", marker=dict(size=10, color="dimgray")), row=3, col=1)

# OPTIONAL: pre-add a few soft gradient rings as SHAPES (not traces) so we only update x0..y1
# (kept minimal here; extend if you like)
fig.update_layout(shapes=[
    dict(type="circle", xref="x2", yref="y2", x0=0, y0=0, x1=0, y1=0, fillcolor="rgba(255,215,0,0.8)", line=dict(width=0), layer="below"),  # solar glow
    dict(type="circle", xref="x3", yref="y3", x0=0, y0=0, x1=0, y1=0, fillcolor="rgba(192,192,192,0.8)", line=dict(width=0), layer="below"),  # lunar silver
])

# running trails
ex, ey, mx, my = [], [], [], []

frames = []
for i, dt in enumerate(times):
    et = utc_to_et(dt)

    # --- physics (unchanged helpers) ---
    r_es, r_ms, r_sun_geo, r_moon_geo = heliocentric_positions(et)
    d_es = np.linalg.norm(r_sun_geo); d_em = np.linalg.norm(r_moon_geo)

    sunR = get_angular_radius(get_mean_radius_km("SUN"),  d_es) * DEG
    moonR= get_angular_radius(get_mean_radius_km("MOON"), d_em) * DEG
    sep  = get_angle_between_vectors(r_sun_geo, r_moon_geo) * DEG

    frac = circle_overlap_fraction(math.radians(sunR), math.radians(moonR), math.radians(sep))
    cover = 100.0 * frac
    label = get_solar_metrics_at_time(et)["classification"]

    Rm, ru, rp, s = lunar_shadow_at_moon(et)
    lm = get_lunar_metrics_at_time(et)
    umb_pct = lm["umbra_fraction_of_moon"]    * 100.0
    pen_pct = lm["penumbra_fraction_of_moon"] * 100.0

    # update trails
    ex.append(r_es[0]); ey.append(r_es[1])
    mx.append(r_ms[0]); my.append(r_ms[1])

    # bottom ring using precomputed cos/sin
    r_geo = np.linalg.norm(r_moon_geo)
    ring_x = r_geo * RCOS; ring_y = r_geo * RSIN

    # marker sizes (proportional; tune constants only once)
    sun_op = 1.0 - 0.75 * (cover/100.0)
    sun_marker  = dict(size=sunR*400,  color=f'rgba(255,215,0,{sun_op:.2f})', line=dict(width=1, color="gold"), symbol='circle')
    moon_marker = dict(size=moonR*400, color='black', line=dict(width=1, color="black"), symbol='circle')

    umb_op = 0.4 + 0.6*(umb_pct/100.0)
    pen_op = 0.2 + 0.4*(pen_pct/100.0)
    pen_marker  = dict(size=rp*0.02, color=f'rgba(120,120,120,{pen_op:.2f})', line=dict(width=0.5, color='gray'),   symbol='circle')
    umb_marker  = dict(size=ru*0.02, color=f'rgba(0,0,0,{umb_op:.2f})',       line=dict(width=0.5, color='black'),  symbol='circle')
    moon_km_mrk = dict(size=Rm*0.02, color='silver',                          line=dict(width=1,   color='dimgray'),symbol='circle')

    solar_text = f"Sun covered: {cover:.1f}%<br>{label}<br>{dt.strftime('UTC %Y-%m-%d %H:%M')}"
    lunar_text = f"Umbra cover: {umb_pct:.1f}%<br>Penumbral cover: {pen_pct:.1f}%"

    # frame updates only touch the traces we created above (by index), no new traces appended
    frame = go.Frame(
        name=str(i),
        data=[
            # row1 system
            go.Scatter(x=[0], y=[0]),                                  # i_sys_sun
            go.Scatter(x=np.array(ex), y=np.array(ey)),                # i_sys_epath
            go.Scatter(x=[ex[-1]], y=[ey[-1]]),                        # i_sys_earth
            go.Scatter(x=np.array(mx), y=np.array(my)),                # i_sys_mpath
            go.Scatter(x=[mx[-1]], y=[my[-1]]),                        # i_sys_moon

            # row2 solar
            go.Scatter(x=[0.0],    y=[0.0],    marker=sun_marker),     # i_sol_sun
            go.Scatter(x=[sep],    y=[0.0],    marker=moon_marker),    # i_sol_moon
            go.Scatter(x=[-1.8],   y=[1.2],    text=[solar_text], mode="text"),  # i_sol_text

            # row2 lunar (Moon fixed at origin; shadow centers at (s,0))
            go.Scatter(x=[s],      y=[0.0],    marker=pen_marker),     # i_lun_pen
            go.Scatter(x=[s],      y=[0.0],    marker=umb_marker),     # i_lun_umb
            go.Scatter(x=[0.0],    y=[0.0],    marker=moon_km_mrk),    # i_lun_moon
            go.Scatter(x=[-LIM_LUNAR*0.9], y=[LIM_LUNAR*0.9], text=[lunar_text], mode="text"),  # i_lun_text

            # row3 geo ring + earth + moon
            go.Scatter(x=ring_x,   y=ring_y),                          # i_geo_ring
            go.Scatter(x=[0],      y=[0]),                              # i_geo_earth
            go.Scatter(x=[r_moon_geo[0]], y=[r_moon_geo[1]]),          # i_geo_moon
        ],
        traces=[  # explicitly bind which traces these updates affect (stable indices)
            i_sys_sun, i_sys_epath, i_sys_earth, i_sys_mpath, i_sys_moon,
            i_sol_sun, i_sol_moon, i_sol_text,
            i_lun_pen, i_lun_umb, i_lun_moon, i_lun_text,
            i_geo_ring, i_geo_earth, i_geo_moon
        ],
        # # OPTIONAL: update shapes each frame instead of adding gradient traces
        # layout=go.Layout(shapes=[
        #     dict(type="circle", xref="x2", yref="y2",  # solar glow follows Sun radius
        #          x0, y0, x1, y1 := bounds_from_center_r(0.0, 0.0, sunR),
        #          fillcolor=f"rgba(255,215,0,{0.55 + 0.45*sun_op:.2f})", line=dict(width=0), layer="below"),
        #     dict(type="circle", xref="x3", yref="y3",  # lunar silver around Moon
        #          x0, y0, x1, y1 := bounds_from_center_r(0.0, 0.0, Rm),
        #          fillcolor="rgba(192,192,192,0.6)", line=dict(width=0), layer="below"),
        # ])
    )
    frames.append(frame)

# ---------- controls ----------
sliders = [dict(
    steps=[dict(method="animate",
                args=[[str(k)], dict(mode="immediate",
                                     frame=dict(duration=120, redraw=True),
                                     transition=dict(duration=0))],
                label=times[k].strftime("%Y-%m-%d")) for k in range(N)],
    transition=dict(duration=0),
    x=0.10, y=0.02, xanchor="left", yanchor="top",
    len=0.80, currentvalue=dict(prefix="Time: ", visible=True)
)]
updatemenus = [dict(
    type="buttons", direction="left",
    x=0.10, y=1.06, xanchor="left", yanchor="top",
    buttons=[
        dict(label="Play",  method="animate",
             args=[None, dict(frame=dict(duration=120, redraw=True),
                              transition=dict(duration=0),
                              fromcurrent=True, mode="immediate")]),
        dict(label="Pause", method="animate",
             args=[[None], dict(frame=dict(duration=0, redraw=False),
                                transition=dict(duration=0),
                                mode="immediate")])
    ]
)]

fig.update_layout(
    title="Orbital Motion + Solar & Lunar Eclipses (rows split; disks stay circular)",
    height=950, sliders=sliders, updatemenus=updatemenus, showlegend=True
)

fig.update(frames=frames)
fig.show()
